<a href="https://colab.research.google.com/github/aa-jesse/Machine-Learning-Projects/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dependencies

In [3]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Processing 

In [4]:
movies_data = pd.read_csv('/movies.csv')

In [5]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [9]:
#sorting the features to the needed feature for the recommendation
selected_features = ['genres', 'director', 'tagline', 'keywords', 'cast', ]
print(selected_features)

['genres', 'director', 'tagline', 'keywords', 'cast', 'keywords']


In [8]:
#find the number of features in the data
movies_data.shape

(4803, 24)

In [11]:
#filling the spaces with string space
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [14]:
#combine selected feature to create newdataframe
combined_features = movies_data['genres']+''+movies_data['director']+''+movies_data['tagline']+''+movies_data['cast']+''+movies_data['keywords']

In [15]:
print(combined_features)

0       Action Adventure Fantasy Science FictionJames ...
1       Adventure Fantasy ActionGore VerbinskiAt the e...
2       Action Adventure CrimeSam MendesA Plan No One ...
3       Action Crime Drama ThrillerChristopher NolanTh...
4       Action Adventure Science FictionAndrew Stanton...
                              ...                        
4798    Action Crime ThrillerRobert RodriguezHe didn't...
4799    Comedy RomanceEdward BurnsA newlywed couple's ...
4800    Comedy Drama Romance TV MovieScott SmithEric M...
4801    Daniel HsiaA New Yorker in ShanghaiDaniel Henn...
4802    DocumentaryBrian HerzlingerDrew Barrymore Bria...
Length: 4803, dtype: object


In [16]:
#convert text to feature vectors for cosine similarity

vectorizer = TfidfVectorizer()

In [18]:
feature_vector = vectorizer.fit_transform(combined_features)

In [19]:
print(feature_vector)

  (0, 22056)	0.20680448381774166
  (0, 4524)	0.24129723125747335
  (0, 25391)	0.12878673509370508
  (0, 22207)	0.33494384233173136
  (0, 9354)	0.1632794825334368
  (0, 4301)	0.21469491540317662
  (0, 19943)	0.27578997869720506
  (0, 16015)	0.15806810497585475
  (0, 13791)	0.24129723125747335
  (0, 22522)	0.16284641840192018
  (0, 25611)	0.19525915695491314
  (0, 21728)	0.2034965163990346
  (0, 20684)	0.21469491540317662
  (0, 26674)	0.19910845936585042
  (0, 26278)	0.23645741604532908
  (0, 20724)	0.1577008884885115
  (0, 17861)	0.2630597318996258
  (0, 17471)	0.08752146071262969
  (0, 26263)	0.12627366207544977
  (0, 23370)	0.07568907674840535
  (0, 3427)	0.27578997869720506
  (0, 8619)	0.2285669844598941
  (0, 21072)	0.10025170751020894
  (0, 8237)	0.1186828191348406
  (0, 369)	0.09169159266436624
  :	:
  (4801, 11764)	0.3098428245906111
  (4801, 26642)	0.2955407254203285
  (4801, 7536)	0.25678902622841965
  (4801, 593)	0.19046415498534033
  (4801, 11017)	0.2955407254203285
  (4801, 

Find the Cosine Similarity Confidence Value

In [20]:
similarity = cosine_similarity(feature_vector)

In [21]:
print(similarity)

[[1.         0.07019397 0.01484531 ... 0.         0.         0.        ]
 [0.07019397 1.         0.02817016 ... 0.01208679 0.         0.        ]
 [0.01484531 0.02817016 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01208679 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [22]:
similarity.shape

(4803, 4803)

In [23]:
#user input movie name
movie_name = input('Enter the name of your favorite movie: ')

Enter the name of your favorite movie: avatar


In [24]:
#create a list of all the movie name in the dtaframe

list_of_all_titles = movies_data['original_title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [30]:
#find close matches to the movie name given by the user

find_the_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print (find_the_close_match)

['Avatar']


In [31]:
close_match = find_the_close_match[0]
print(close_match)

Avatar


In [35]:
#find the index of the chosen movie

close_match_index = movies_data[movies_data.title == close_match]['index'].values[0]
print(close_match_index)

0


In [36]:
#get list of similar movies 

similarity_score = list(enumerate(similarity[close_match_index]))
print(similarity_score)

[(0, 1.0000000000000002), (1, 0.07019396978101682), (2, 0.01484531361335894), (3, 0.006880717448307476), (4, 0.09412049317194174), (5, 0.028626008272009333), (6, 0.008614411842771947), (7, 0.024986508233273134), (8, 0.031703569278025014), (9, 0.015080150800416138), (10, 0.0688330887738475), (11, 0.013609439031477918), (12, 0.023012546507706646), (13, 0.019721028629548124), (14, 0.060393844046563706), (15, 0.008139598527303423), (16, 0.015689672184937286), (17, 0.014301974678081336), (18, 0.04801888635863928), (19, 0.02939963066750157), (20, 0.02162635947461574), (21, 0.0054658012310303435), (22, 0.020536382896339654), (23, 0.03848723522610428), (24, 0.035728065052885886), (25, 0.0), (26, 0.06242424980708835), (27, 0.02231840194290098), (28, 0.02425364118731952), (29, 0.01525841683916546), (30, 0.030034085800108044), (31, 0.051874448795608125), (32, 0.025523423532911035), (33, 0.025165424064809955), (34, 0.0), (35, 0.01711308079558758), (36, 0.03145296738949234), (37, 0.0490934661323812

In [39]:
#sort similarity score of movies in descending order

sorted_similar_movies = sorted(similarity_score, reverse =True, key = lambda x:x[1])
print(sorted_similar_movies)

[(0, 1.0000000000000002), (3158, 0.22859468585715392), (94, 0.20325911365496666), (2696, 0.19098590544262753), (47, 0.18360571188355565), (2403, 0.16778556391605937), (1531, 0.1591762844885764), (1951, 0.15449410505452874), (838, 0.1518484499562104), (206, 0.1454641967147878), (1759, 0.14078611133849248), (56, 0.13323288914796375), (1099, 0.1293980146165027), (1473, 0.12480329149112557), (1053, 0.12304928557448656), (43, 0.11982710174100202), (3439, 0.11973561608392866), (4108, 0.11807390367173047), (1650, 0.11738374662278395), (3730, 0.11667393765774632), (239, 0.11501768073727323), (278, 0.11499335634424673), (942, 0.11424642558909634), (3105, 0.1139935734656308), (1275, 0.1125199142166098), (1873, 0.11044628229982545), (812, 0.10988099704851781), (2060, 0.1088716455828997), (1804, 0.10803739155568466), (3056, 0.10403361996032494), (2198, 0.10374250997830638), (305, 0.10340876893943325), (158, 0.10234638024483114), (977, 0.10214541202113091), (2827, 0.10081831965187434), (3208, 0.100

Recommend Movie to **User**

In [41]:
from os import supports_effective_ids
#recommend the names of similar movies based on thier index

print('movies suggested for You: \n')

i = 1 

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

movies suggested for You: 

1 . Avatar
2 . Alien
3 . Guardians of the Galaxy
4 . Jason X
5 . Star Trek Into Darkness
6 . Aliens
7 . Moonraker
8 . Space Dogs
9 . Alien³
10 . Clash of the Titans
11 . The Right Stuff
12 . Star Trek Beyond
13 . Shadow Conspiracy
14 . The Astronaut's Wife
15 . Galaxy Quest
16 . Terminator Salvation
17 . The Terminator
18 . In the Shadow of the Moon
19 . Wing Commander
20 . Cargo
21 . Gravity
22 . Planet of the Apes
23 . The Book of Life
24 . Imaginary Heroes
25 . Sunshine
26 . Blood Ties
27 . Pocahontas
28 . Out of the Furnace
29 . Snow White: A Tale of Terror


Movie Recommendation System

In [48]:
#user input movie name
movie_name = input('Enter the name of your favorite movie: ')

#create a list of all the movie name in the dtaframe
list_of_all_titles = movies_data['original_title'].tolist()

#find close matches to the movie name given by the user
find_the_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_the_close_match[0]

close_match_index = movies_data[movies_data.title == close_match]['index'].values[0]

#get list of similar movies 

similarity_score = list(enumerate(similarity[close_match_index]))

#sort similarity score of movies in descending order
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1], reverse =True, )

print('movies suggested for You: \n')

i = 1 

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Enter the name of your favorite movie: Ant man
movies suggested for You: 

1 . Ant-Man
2 . X-Men
3 . X-Men: Apocalypse
4 . The Avengers
5 . Iron Man 2
6 . Avengers: Age of Ultron
7 . X2
8 . The Incredible Hulk
9 . X-Men: The Last Stand
10 . Deadpool
11 . Thor: The Dark World
12 . X-Men: Days of Future Past
13 . Real Steel
14 . The Image Revolution
15 . The Amazing Spider-Man 2
16 . Man of Steel
17 . Superman II
18 . The Helix... Loaded
19 . Batman v Superman: Dawn of Justice
20 . X-Men: First Class
21 . The Lost Boys
22 . Steel
23 . Spawn
24 . 2 Guns
25 . Kick-Ass 2
26 . Kick-Ass
27 . Whiteout
28 . Teenage Mutant Ninja Turtles: Out of the Shadows
29 . The Brass Teapot


Saving the Trained Model